**Finetuning Lllama 3 8b for text to sql queries**

In [ ]:
%%capture
# Installing Unsloth, Xformers (for speed)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# 1. Load Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.12.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset

# 1. Load the Gretel dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

# 2. Slicing it for speed!
# We shuffle it first 5000 entries to get a mix of domains (Healthcare, Finance, etc.)
dataset = dataset.shuffle(seed=3407).select(range(5000))

# 3. Define the Prompt Format
sql_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    # Map Gretel's specific column names to our prompt
    instructions = examples["sql_prompt"]
    inputs       = examples["sql_context"]
    outputs      = examples["sql"]

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = sql_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 4. Map the formatting function
dataset = dataset.map(formatting_prompts_func, batched = True,)

print(f"Success! Prepared {len(dataset)} examples for training.")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Success! Prepared 5000 examples for training.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # This matches the "text" field we created in Step 3
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Quick demo run. Change to 100 or 200 for a smarter model later!
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # <--- KEEPS W&B DISABLED
    ),
)

trainer.train()

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.033000
2,1.541100
3,1.241600
4,1.331600
5,1.565900
6,1.261400
7,1.248000
8,1.300600
9,1.330600
10,1.332300


TrainOutput(global_step=60, training_loss=0.8586821506420771, metrics={'train_runtime': 378.4288, 'train_samples_per_second': 1.268, 'train_steps_per_second': 0.159, 'total_flos': 4351523922739200.0, 'train_loss': 0.8586821506420771, 'epoch': 0.096})

In [ ]:
# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    sql_prompt.format(
        "Show me all users who signed up last week.", # Instruction
        "CREATE TABLE users (username VARCHAR, email VARCHAR, signup_date DATE)", # Input (Schema)
        "", # Output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Show me all users who signed up last week.

### Input:
CREATE TABLE users (username VARCHAR, email VARCHAR, signup_date DATE)

### Response:
SELECT * FROM users WHERE DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) <= signup_date;<|end_of_text|>


In [ ]:
inputs = tokenizer(
[
    sql_prompt.format(
        "List all product names and their total sales amount.", # Instruction
        "CREATE TABLE products (product_id INT, name TEXT); CREATE TABLE sales (sale_id INT, product_id INT, amount INT)", # Input (2 tables!)
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

NameError: name 'tokenizer' is not defined

In [ ]:
from huggingface_hub import login

# 1. Log in (Paste the token you just created when asked)
login()

# 2. Upload to Hugging Face
model.push_to_hub_gguf(
    "Sanschinu/Llama-3-Text-to-SQL", # Replace "your-username" with your actual HF username
    tokenizer,
    quantization_method = "q8_0",
    token = True,
)

Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [11:52<00:00, 178.17s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [06:12<00:00, 93.11s/it]


Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_9osladgd`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3-8b.F16.gguf']
Unsloth: [2] Converting GGUF f16 into q8_0. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['llama-3-8b.Q8_0.gguf']
Unsloth: No Ollama template mapping found for model 'unsloth/llama-3-8b'. Skipping Ollama Modelfile
U

RuntimeError: Failed to upload to Hugging Face Hub: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-695098ba-0dc001064be09b774580c3fe;ae466db5-f63b-4c31-ab69-f37789ff9af2)

Invalid username or password.

In [ ]:
# 1. Mount Google Drive so we can save the file there
from google.colab import drive
drive.mount('/content/drive')

# 2. Save to 8-bit GGUF (Compatible with Ollama / LM Studio)
model.save_pretrained_gguf("model_quantized", tokenizer, quantization_method = "q8_0")

# 3. Copy it to your Drive (It will be approx 8GB)
!cp -r model_quantized/unsloth.Q8_0.gguf "/content/drive/MyDrive/Text-to-SQL-Llama3.gguf"

Mounted at /content/drive
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [06:22<00:00, 95.51s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [03:42<00:00, 55.54s/it]


Unsloth: Merge process complete. Saved to `/content/model_quantized`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['llama-3-8b.F16.gguf']
Unsloth: [2] Converting GGUF f16 into q8_0. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['llama-3-8b.Q8_0.gguf']
Unsloth: No Ollama template mapping found for model 'unsloth/llama-3-8b'. Skipping Ollama Modelfile
Uns